# Colaboratoryで実行する場合
以下を実行して、外部ファイルをダウンロードしてください。   
**このセルはColaboratoryを起動するたびに必要となります**

In [ ]:
##################################
### Colaboratoryのみ以下を実行 ###
##################################
import sys
if 'google.colab' in sys.modules:
    !wget -P ./text http://www.hal.t.u-tokyo.ac.jp/~yamakata/lecture/mediaproc/mediaproc4/mediaproc4-1.zip
    !unzip text/mediaproc4-1.zip -d text


# トピック分析1: Bag-of-Words (BoW)による文書のベクトル表現

BoWとは、その文書が、どのような単語の集合から構成されているかだけに注目して、その文書の特徴をベクトル表現に変換する手法です。   
つまりBoWでは単語の並びは考慮しません。

## 1. 文書間の類似度

下のセルでtextに代入された８文のそれぞれが（非常に短いですが）一つの文書と考えて、それぞれの文書の文書ベクトルを算出しましょう。

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# たった5, 6単語からなる文書ですが、以下の各列を1つの文書と考えましょう
text = ['私 は 本 を 読む 。',
        '今日 は 晴天 だ 。',
        '私 は 私 だ 。',
        '本 は 本 で 本 だ 。',
        '私 は 本 を 読む 。',
        '私 本 を 読む 。',
        '私 は 本 を 読む 。 本 を 私 は 読む 。',
        '明日 雨 が 降る ！']

# BoWを作成する準備
# token_patternは、文書中で1単語がどう書かれているかのパターンを指す
# ここではスペース以外の文字が1文字以上続いたものを1単語とする（スペースが来たらそこで単語終わり）
count = CountVectorizer(token_pattern=r'[^\s]+')
# 文書からbowを取り出す
bow = count.fit_transform(text)

# 語彙サイズ
print('語彙サイズ:', len(count.get_feature_names()))
 
print('---- 語彙リストを出力する。数字はその単語のID ----')
print(sorted(count.vocabulary_.items(), key=lambda x:x[1]))
# 語彙サイズは15（0～14)であることから、文書ベクトルの次元は15次元となる

print('---- 各文書の文書ベクトル ----')
# 例えば1文目　'私 は 本 を 読む 。'は「私」「は」「本」「を」「読む」「。」の6個の単語から構成されるため、
# そのそれぞれの単語のIDの値が1になり、文書に現れていないIDの値は0になっている
vec = bow.toarray()

for i in range(len(text)):
    print(text[i], ':\t', vec[i])


この８つの文書の語彙は合わせて15種類(IDは0～14)なので、各文書は15次元の文書ベクトルに変換されます。   
ある文書の文書ベクトルとは、その文書中に各語彙が何回現れるかを数えてベクトル化したものです。   
各文書と、各語彙の出現回数は以下の表のようになります。

| 文書 | 0=。| 1=が | 2=だ | 3=で | 4=は | 5=を | 6=今日 | 7=明日 | 8=晴天 | 9=本 | 10=私 | 11=読む | 12=降る | 13=雨 | 14=！ |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |
| 「私 は 本 を 読む 。」 | 1 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0|
| 「今日 は 晴天 だ 。」 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0|
| 「私 は 私 だ 。」 | 1 | 0 | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0|
| 「本は 本 で 本 だ 。」 | 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 3 | 0 | 0 | 0 | 0 | 0|
| 「私 は 本 を 読む 。」| 1 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0|
| 「私 本 を 読む 。」| 1 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0|
| 「私 は 本 を 読む 。 本 を 私 は 読む 。」| 2 | 0 | 0 | 0 | 2 | 2 | 0 | 0 | 0 | 2 | 2 | 2 | 0 | 0 | 0 |
| 「明日 雨 が 降る ！」 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 1 | 1 | 1|

「私 は 私 だ 。」	は「私」が2回でているので、「10=私」の欄の値が2になっていますね。   
また、「私 は 本 を 読む 。」と「私 本 を 読む 。」の文書ベクトルを比べると、「4=は」のみが異なり、
それ以外は全く同じであることが分かります。
「私 は 本 を 読む 。」に比べて「私 は 本 を 読む 。 本 を 私 は 読む 。」はすべての単語が2回ずつ現れているので、ベクトルの要素の値はすべて倍になっています。

## 2. コサイン距離によるベクトル間の類似度

文書間の類似度は、この文書ベクトル同士がどれだけ似通っているかで判断することができそうです。   
ベクトル間の類似度の尺度はいろいろありますが、中でもよくつかわれるのがコサイン類似度です。   
これは、二つのベクトルのなす角を$\theta$とすると、$\cos (\theta)$となります。   
通常、ベクトル間のコサイン類似度は$-1$から$1$までの値をとりますが、文書ベクトルは通常、正の要素しか持たないのでベクトルの向きが逆になることはないことから、文書ベクトル間のコサイン類似度は$0$から$1$までとなります。

ベクトル$V_1$とベクトル$V_2$の類似度$\mbox{cos_sim}$は以下のように計算できます。
$$\mbox{cos_sim}(V_1, V_2) = \frac{V_1 \cdot V_2}{|V_1||V_2|}$$

ベクトルの内積で表しているのでわかりづらいかもしれませんが、高校で習った式と同じです。   
二つの直線$ax+by=0$，$cx+dy=0$のなす角$\theta$を求めるには、  
$$\cos (\theta) = \frac{|ac+bd|}{\sqrt{a^2 + b^2}\sqrt{c^2 + d^2}}$$
でしたね。   
$ax+by=0$の法線ベクトルは$(a,b)$であること考慮すると、これはベクトル$(a, b)$とベクトル$(c, d)$のなす角を計算していることになっています。

In [ ]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

これを使って、文書間のコサイン類似度を計算してみましょう。   


In [ ]:
# 1文目と2～4文目はどの程度近いかを、文書ベクトルのコサイン類似度で評価してみよう
for i in range(1, len(text)):
    sim = cos_sim(vec[0], vec[i])
    print('「', text[0], '」と「', text[i], '」の類似度:\t', round(sim, 3))

「 私 は 本 を 読む 。 」と「 私 本 を 読む 。 」は1単語違うだけなので、0.913という高い値を示しています。   
一方で「 私 は 本 を 読む 。 」と「 今日 は 晴天 だ 。 」では、一致している単語が「は」と「。」しかありませんので、0.365と低い値を示します。   
「 私 は 本 を 読む 。 」と「 私 は 本 を 読む 。 」は完全に同じ文書なので類似度は1.0となります。  
また、「 私 は 本 を 読む 。 」と「 私 は 本 を 読む 。 本 を 私 は 読む 。 」とは、同じ文書ではありませんが、
後者のベクトルの長さが前者の2倍になっただけで、ベクトルのなす角は0ですから、類似度はやはり1.0となります。

## 3. 小説間の類似度

今度はもう少し長い文書の類似度を計算してみましょう。

宮沢賢治の「銀河鉄道の夜」「風の又三郎」と太宰治の「人間失格」の3作品について、
お互いがどの程度近いかを文書ベクトルのコサイン類似度で評価してみましょう。   

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

titles = ['銀河鉄道の夜', '風の又三郎', '人間失格']

# 以下に3作品の分かち書き文が入っています
files = ['text/gingatetsudonoyoru_wakati.txt', 'text/kazenomatasaburo_wakati.txt', 'text/ningenshikkaku_wakati.txt']

# 3つの文書を読み込んで一つのリスト`wakati_all`にまとめます
wakati_all = []
for i in range(len(files)):
    with open(files[i], 'r', encoding='utf-8') as f:
        wakati_all.append(f.read().replace('\n', ' '))

# BoWを作成する準備
# token_patternは、文書中で1単語がどう書かれているかのパターンを指す
# ここではスペース以外の文字が1文字以上続いたものを1単語とする（スペースが来たらそこで単語終わり）
novel_count = CountVectorizer(token_pattern=r'[^\s]+')
# 文書からbowを取り出す
novel_bow = novel_count.fit_transform(wakati_all)
novel_vec = novel_bow.toarray()


これで、`novel_vec`に各小説の文書ベクトルが代入されました。   
それでは、3つの小説が互いにどの程度近いのかを、類似度行列として出力してみましょう。


In [ ]:
# 語彙リスト
#print('語彙リスト:', novel_count.get_feature_names())
 
# 語彙サイズ
print('語彙サイズ:', len(novel_count.get_feature_names()))
 
# 「銀河鉄道の夜」「風の又三郎」「人間失格」のconfusion matrixを出力してみましょう
novel_sim = []
for i in range(len(files)):
    novel_sim.append([])
    for j in range(len(files)):  
        novel_sim[i].append(cos_sim(novel_vec[i], novel_vec[j]))
print('「銀河鉄道の夜」,「風の又三郎」,「人間失格」')
print(np.array(novel_sim))


これを見ると、「銀河鉄道の夜」と「風の又三郎」は0.94と似ていますが、「銀河鉄道の夜」と「人間失格」とは0.88と相対的に似ていないことが分かります。   
「人間失格」には、「風の又三郎」よりも「銀河鉄道の夜」のほうが似ているようですね。


本課題の作品データは[青空文庫](https://www.aozora.gr.jp/index.html)のものを使用しています。  
ただし、ルビや入力者注、アクセント分解された欧文や編者による注記等は削除しました。   